In [57]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

# WRANGLE

In [58]:
import pandas as pd

df = pd.read_csv(DATA_PATH+'burritos/burritos.csv',
                parse_dates=['Date'],
                index_col=['Date'])

In [59]:
df.shape

(423, 65)

In [60]:
#df.info()

In [61]:
df.head()

,Location,Burrito,Neighborhood,Address,URL,Yelp,Google,Chips,Cost,Hunger,...,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-18,Donato's taco shop,California,Miramar,6780 Miramar Rd,http://donatostacoshop.net/,3.5,4.2,NaN,6.49,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-24,Oscar's Mexican food,California,San Marcos,225 S Rancho Santa Fe Rd,http://www.yelp.com/biz/oscars-mexican-food-sa...,3.5,3.3,NaN,5.45,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-24,Oscar's Mexican food,Carnitas,NaN,NaN,NaN,NaN,NaN,NaN,4.85,1.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-24,Oscar's Mexican food,Carne asada,NaN,NaN,NaN,NaN,NaN,NaN,5.25,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-27,Pollos Maria,California,Carlsbad,3055 Harding St,http://pollosmaria.com/,4.0,3.8,x,6.59,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Missing values (impute)
- High Cardinality
- Weird Values eg. Y/N/X/x/0/1/T/F

- What do we want to predict?
    - `overall`

In [62]:
df.select_dtypes('object').nunique().sort_values()

Zucchini           1
Carrots            1
Unreliable         1
Chile relleno      1
Nopales            1
Lobster            1
Ham                1
Mushroom           1
Bacon              1
Sushi              1
Avocado            1
Egg                1
Tomato             2
Pineapple          2
Taquito            2
Onion              2
Cilantro           2
Salsa.1            2
Sauce              2
Cabbage            2
Bell peper         2
Lettuce            2
Beans              2
Rice               2
NonSD              2
Corn               2
Pico               2
Guac               2
Cheese             2
Beef               2
Sour cream         2
Pork               2
Chicken            2
Shrimp             2
Fish               2
Fries              2
Chips              4
Rec                6
Neighborhood      41
URL               86
Address           87
Reviewer         106
Location         108
Burrito          132
Notes            145
dtype: int64

In [63]:
## Drop rows with null target values
df['overall'].isnull().sum()

2

In [66]:
def wrangle(df):
    df = df.copy()
    
    ## Drop null target values
    df.dropna(subset=['overall'], inplace=True)
    
    ## Create target column
    df['great'] = (df['overall'] >= 4).astype(int)
    
    ## Drop original 'overall' column to prevent leakage
    df.drop(columns='overall', inplace=True)
    
    ## Fix binary columns
    categorical_cols = df.select_dtypes('object').nunique()
    binary_cols = [col for col in categorical_cols.index if categorical_cols[col] <= 2]
    for col in binary_cols:
        df[col] = df[col].apply(lambda x: 1 if isinstance(x, str) else 0)
        
    ## Fix burrito types
    burrito_cats = ['california', 'asada', 'carnitas', 'surf', 'pastor']
    for cat in burrito_cats:
        df[cat] = df['Burrito'].str.lower().str.strip().str.contains(cat).astype(int)
    df.drop(columns='Burrito', inplace=True)
    
    threshold = 10
    high_card_cols = [col for col in categorical_cols.index if categorical_cols[col] > threshold]
    df.drop(high_card_cols, axis=1, inplace=True)
    
    ## Drop columns with high NaN value counts
    df.dropna(axis=1, thresh=300, inplace=True)
    
    return df

In [67]:
df2 = wrangle(df)

KeyError: "['Burrito'] not found in axis"

In [ ]:
df2.head()

# SPLIT

# BASELINE

In [ ]:
print('Baseline Accuracy:', y_train.value_counts(normalize=True))